Below is a picture about how this is being formulated in 3D. </br>
<img src='./1.png' width="500" height="600"/> </br>

For 2D, there are two cases: TEz mode and TMz mode which stands for if the E field is parallel to the page or the H field is parallel to the page. We can just implement one of them and the other will directly follows.
First, we see the two important Maxwell equation going to be used in this method:
$$
\begin{array}{l}
\frac{\partial \vec{H}}{\partial t}=-\frac{1}{\mu} \nabla \times \vec{E}-\frac{\rho^{\prime}}{\mu} \vec{H} \\
\frac{\partial \vec{E}}{\partial t}=\frac{1}{\varepsilon} \nabla \times \vec{H}-\frac{\sigma}{\varepsilon} \vec{E}
\end{array}
$$
These are the curl equations we'll use and from this we can derive the six equations for TEz and TMz mode:</br>
__TMz mode__
$$
\begin{split}
        \frac{\partial H_x}{\partial t} &= \frac{1}{\mu}\left[-\frac{\partial E_z}{\partial y} - \left( M_x + \sigma^*H_x\right)\right]\\
        \frac{\partial H_y}{\partial t} &= \frac{1}{\mu}\left[\frac{\partial E_z}{\partial x} - \left( M_y + \sigma^*H_y\right)\right]\\
        \frac{\partial E_z}{\partial t} &= \frac{1}{\varepsilon}\left[\frac{\partial H_y}{\partial x} - \frac{\partial H_x}{\partial y} - \left( J_z + \sigma E_z\right)\right]
\end{split}
$$
__TEz mode__
$$
    \begin{split}
        \frac{\partial E_x}{\partial t} &= \frac{1}{\varepsilon}\left[\frac{\partial H_z}{\partial y} - \left( J_x + \sigma E_x\right)\right]\\
        \frac{\partial E_y}{\partial t} &= \frac{1}{\varepsilon}\left[-\frac{\partial H_z}{\partial x} - \left( J_y + \sigma E_y\right)\right]\\
        \frac{\partial H_z}{\partial t} &= \frac{1}{\mu}\left[\frac{\partial E_x}{\partial y} - \frac{\partial E_y}{\partial x} - \left( M_z + \sigma^* H_z\right)\right]
    \end{split}
$$
If we combine forward euler and leap frog method that consider either H or E as a middle time step in a stagger grid with same $\Delta$ or resolution for simplicity, we can derive the following update formulas for TMz mode and since we don't implement TEz here, we ignore that as they should be very similar:
$$
Hx(m, n) = Chh(m, n) * Hx(m, n) - Chxe(m, n) * (Ez(m, n + 1) - Ez(m, n)); \\
Hy(m, n) = Chh(m, n) * Hy(m, n) + Chye(m, n) * (Ez(m + 1, n) - Ez(m, n)); \\ 
Ez(m, n) = Cee(m, n) * Ez(m, n) + Ceh(m, n) * ((Hy(m, n) - Hy(m - 1, n)) - (Hx(m, n) - Hx(m, n - 1)));\\
$$
Notice that we assume it to be lossless material and non permiable, therefore, we have Chh and Cee to be 1.
Here, Chxe, Chye and Ceh are the update coefficients. Since the only different for Chxe and Chye is the resolution and position, we call it Che here and later in the implementation, it need to splited. 
$$
    \begin{split}
Ceh = \frac{1}{\varepsilon}\frac{\Delta_{t}}{\Delta_x}
& = \frac{1}{\varepsilon_r\varepsilon_0}\frac{n\sqrt{\varepsilon_0\mu_0}}{n\sqrt{\varepsilon_0\mu_0}}\frac{\Delta_{t}}{\Delta_x}
 = \frac{n\sqrt{\varepsilon_0\mu_0}}{\varepsilon_r\varepsilon_0}\frac{c\Delta_{t}}{\Delta_x}
 = \frac{\sqrt{\varepsilon_0\mu_0}}{\varepsilon_r\varepsilon_0}\frac{c_0\Delta_{t}}{\Delta_x}
 = \frac{\eta_0}{\varepsilon_r}\bar{S}_x \\
Che = \frac{1}{\mu}\frac{\Delta_{t}}{\Delta_x} &=
\frac{1}{\mu_r\mu_0}\frac{n\sqrt{\varepsilon_0\mu_0}}{n\sqrt{\varepsilon_0\mu_0}}\frac{\Delta_{t}}{\Delta_x}
 = \frac{n\sqrt{\varepsilon_0\mu_0}}{\mu_r\mu_0}\frac{c\Delta_{t}}{\Delta_x}
 = \frac{\sqrt{\varepsilon_0\mu_0}}{\mu_r\mu_0}\frac{c_0\Delta_{t}}{\Delta_x}
 = \frac{1}{\eta_0\mu_r}\bar{S}_x,
    \end{split}
$$
where $\eta_0 = \sqrt{\mu_0/\varepsilon_0}$ is the free-space impedance, 
$n = \sqrt{\varepsilon_r\mu_r}$ is the refractive index, 
$c_0$ is the speed of light in free space,
and $\bar{S}_x$ is the Courant number along $x$-direction in free space as we use the same resolution for x and y. \\
Now we have all the equations needed and finally, we need to choose dt for updates. We just use the simple formula 
$$
\Delta t = \frac{1}{\sqrt{\frac{1}{\Delta_x^2} + \frac{1}{\Delta_y^2}}}
$$
We also may want to see why this works, and here is a illustration for this setting satisfing Ampere's Law. Forgive my awful handwritting and drawing...
<img src='./2.jpeg' width="500" height="600"/> </br>


Because we use staggered grid, the number of data for the center field and the field on edge is different. This can be shown using the image attached below:
<img src='./3.jpeg' width="500" height="600"/> </br>